In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pydot as pydot
import math

/var/folders/6k/rf2_mhpd7790drjk7smpzn2r0000gn/T/ipykernel_40579/2058174243.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Load the dataset
columns = [
    'Family Name', 
    'Variation',
    'Algorithm Name', 
    'Year',
    'Looked at?',
    'Relevant?',
    'Parallel?',
    'Time Complexity Class',
    'Approximation Factor Class (Mult)', 
    'Approximation Factor Class (Add)',
    'Approximation Type (additive/multiplicative)',	
    'Parametrized Approximation?',
    'Type of PTAS (if PTAS)',
    'Quantum?',
    'Time Encoding',
    'Approx Error "Size" (for figures only)'
]
approximation_algorithms = pd.read_csv('approximation_algorithms.csv', usecols=columns)

# Clean and prepare the data
approximation_algorithms = approximation_algorithms[approximation_algorithms['Looked at?'] != '0.001']
approximation_algorithms = approximation_algorithms[approximation_algorithms['Looked at?'] != '0']
approximation_algorithms = approximation_algorithms[approximation_algorithms['Relevant?'] != 0]
approximation_algorithms = approximation_algorithms[approximation_algorithms['Relevant?'] != '0']

approximation_algorithms = approximation_algorithms[approximation_algorithms['Parallel?'] != '1']

approximation_algorithms = approximation_algorithms[approximation_algorithms['Quantum?'] != '1']
approximation_algorithms = approximation_algorithms[approximation_algorithms['Quantum?'] != 1]






approximation_algorithms['Year'] = pd.to_numeric(approximation_algorithms['Year'].str.extract(r'(\d{4})')[0], errors='coerce')
approximation_algorithms.dropna(subset=['Year'], inplace=True)
# approximation_algorithms['Year'] = approximation_algorithms['Year'].astype(int)

# drop empty family names
approximation_algorithms = approximation_algorithms[approximation_algorithms['Family Name'].notnull()]

for col in ['Time Complexity Class', 'Approximation Factor Class (Mult)', 'Approximation Factor Class (Add)']:
    # Extract numeric (float) from string
    approximation_algorithms[col] = approximation_algorithms[col].astype(str).str.extract(r'(\d+\.?\d*)')[0].astype(float)
    approximation_algorithms[col] = approximation_algorithms[col].replace(0, 99999)

# drop empty time complexity classes
approximation_algorithms = approximation_algorithms[approximation_algorithms['Time Complexity Class'].notnull()]

# round time complexity to upper integer
approximation_algorithms['Time Complexity Class'] = np.ceil(approximation_algorithms['Time Complexity Class']).astype(int)



In [3]:
# read sheet1
# Load the dataset
columns = [
    'Family Name',
    'Variation',
    'Algorithm Name',
    'Year',
    'Looked at?',
    'Time Complexity Class',
    'Approximate?',
    'Parallel?',
    'Quantum?',
    'Time Encoding',

]
sheet1 = pd.read_csv('sheet1.csv', usecols=columns)

# Clean and prepare the data
# drop Looked at? == 0.001
sheet1 = sheet1[sheet1['Looked at?'] != 0.001]
# drop Parallel? == 1
sheet1 = sheet1[sheet1['Parallel?'] != '1']
sheet1 = sheet1[sheet1['Quantum?'] != '1']
sheet1 = sheet1[sheet1['Quantum?'] != 1]
# drop approximate? == 1
sheet1 = sheet1[sheet1['Approximate?'] != '1']


# parse time complexity class
sheet1['Time Complexity Class'] = sheet1['Time Complexity Class'].str.extract(
    r'(\d+\.?\d*)')[0].astype(float)


sheet1['Approx Error "Size" (for figures only)'] = '0'

# drop if Famiy Name is not in approximation_algorithms
sheet1 = sheet1[sheet1['Family Name'].isin(
    approximation_algorithms['Family Name'])]

# round time complexity class to upper integer
sheet1['Time Complexity Class'] = np.ceil( sheet1['Time Complexity Class']).astype(int)


In [4]:
# min time complexity class for each family
min_time_complexity_class = approximation_algorithms.groupby(
    'Family Name')['Time Complexity Class'].min()

min_exact_time_complexity_class = sheet1.groupby(
    'Family Name')['Time Complexity Class'].min()

min_exact_time_complexity_class

time_improvement = pd.DataFrame()
time_improvement['Family Name'] = min_exact_time_complexity_class.index
time_improvement['From'] = min_exact_time_complexity_class.values
time_improvement['To'] = min_time_complexity_class.loc[min_exact_time_complexity_class.index].values

n_family = len(time_improvement)

time_improvement = time_improvement[time_improvement['From']
                                    != time_improvement['To']]

count_improvements = time_improvement.groupby(['From', 'To']).size()

digraph = pydot.Dot(graph_type='digraph', rankdir='LR',
                    splines=True, nodesep=5, ranksep=1,
                    concentrate=True, overlap=False)


digraph.set_node_defaults(shape='circle', fontsize='10', fontname='Arial', color='black', style='filled', fillcolor='white', penwidth='1', weight='1')
digraph.set_edge_defaults(fontsize='10', fontname='Arial', color='black', arrowsize='0.5', penwidth='1', weight='1', minlen='5')


nodes = {
    1: ('1', '7,0!'),
    2: ('logn', '6,0!'),
    3: ('n', '5,0!'),
    4: ('nlogn', '4,0!'),
    5: ('n^2', '3,0!'),
    6: ('n^3', '2,0!'),
    7: ('>n^3', '1,0!'),
    8: ('n! | c^c', '0,0!')
}

for node_id, (label, pos) in nodes.items():
    node = pydot.Node(str(node_id), label=label, pos=pos, pin=True)
    node.set_pos(pos)
    digraph.add_node(node)


for (from_, to), count in count_improvements.items():
    edge = pydot.Edge(str(from_), str(to), label=f'{count/n_family:.2%}%', penwidth=math.log(count+1))
    digraph.add_edge(edge)

digraph.write_png('time_improvement.png', prog='neato')